In [1]:
import pandas as pd
from sqlalchemy import create_engine, func, inspect
from config import password

#### Extract CSVs into DataFrames

In [2]:
credits_file = "Resources/credits.csv"
credits_df = pd.read_csv(credits_file)
credits_df.head()

,person_id,id,name,character,role
0,21174,tm19248,Buster Keaton,Johnny Gray,ACTOR
1,28713,tm19248,Marion Mack,Annabelle Lee,ACTOR
2,28714,tm19248,Glen Cavender,Captain Anderson,ACTOR
3,28715,tm19248,Jim Farley,General Thatcher,ACTOR
4,27348,tm19248,Frederick Vroom,A Southern General,ACTOR


In [3]:
titles_file = "Resources/titles.csv"
titles_df = pd.read_csv(titles_file)
titles_df.head()

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,tm19248,The General,MOVIE,"During America’s Civil War, Union spies steal ...",1926,NaN,78,"['action', 'drama', 'war', 'western', 'comedy'...",['US'],NaN,tt0017925,8.2,89766.0,8.647,8.0
1,tm83884,His Girl Friday,MOVIE,"Hildy, the journalist former wife of newspaper...",1940,NaN,92,"['comedy', 'drama', 'romance']",['US'],NaN,tt0032599,7.8,57835.0,11.270,7.4
2,tm19424,Detour,MOVIE,"The life of Al Roberts, a pianist in a New Yor...",1945,NaN,66,"['thriller', 'drama', 'crime']",['US'],NaN,tt0037638,7.3,17233.0,7.757,7.2
3,tm112005,Marihuana,MOVIE,A young girl named Burma attends a beach party...,1936,NaN,57,"['crime', 'drama']",['US'],NaN,tt0026683,4.0,864.0,3.748,3.6
4,tm22806,Intolerance: Love's Struggle Throughout the Ages,MOVIE,"The story of a poor young woman, separated by ...",1916,NaN,197,"['history', 'drama']",['US'],NaN,tt0006864,7.7,15242.0,9.412,7.2


### Create new data with select columns

In [4]:
credits_df = credits_df[["id","name","role"]]
credits_df.head()

,id,name,role
0,tm19248,Buster Keaton,ACTOR
1,tm19248,Marion Mack,ACTOR
2,tm19248,Glen Cavender,ACTOR
3,tm19248,Jim Farley,ACTOR
4,tm19248,Frederick Vroom,ACTOR


In [5]:
titles_df = titles_df[["id","title","type","genres","production_countries"]]
titles_df.head()

,id,title,type,genres,production_countries
0,tm19248,The General,MOVIE,"['action', 'drama', 'war', 'western', 'comedy'...",['US']
1,tm83884,His Girl Friday,MOVIE,"['comedy', 'drama', 'romance']",['US']
2,tm19424,Detour,MOVIE,"['thriller', 'drama', 'crime']",['US']
3,tm112005,Marihuana,MOVIE,"['crime', 'drama']",['US']
4,tm22806,Intolerance: Love's Struggle Throughout the Ages,MOVIE,"['history', 'drama']",['US']


 ### Clean DataFrame

In [6]:
credits_df.drop_duplicates(["id","name"], inplace=True)
credits_df

,id,name,role
0,tm19248,Buster Keaton,ACTOR
1,tm19248,Marion Mack,ACTOR
2,tm19248,Glen Cavender,ACTOR
3,tm19248,Jim Farley,ACTOR
4,tm19248,Frederick Vroom,ACTOR
...,...,...,...
39837,tm1045408,Kate Davis,DIRECTOR
39838,tm1045408,David Heilbroner,DIRECTOR
39839,tm1106415,Guillem Balagué,ACTOR
39840,tm1106415,Alex Gale,DIRECTOR


In [7]:
titles_df.drop_duplicates("id", inplace=True)
titles_df

,id,title,type,genres,production_countries
0,tm19248,The General,MOVIE,"['action', 'drama', 'war', 'western', 'comedy'...",['US']
1,tm83884,His Girl Friday,MOVIE,"['comedy', 'drama', 'romance']",['US']
2,tm19424,Detour,MOVIE,"['thriller', 'drama', 'crime']",['US']
3,tm112005,Marihuana,MOVIE,"['crime', 'drama']",['US']
4,tm22806,Intolerance: Love's Struggle Throughout the Ages,MOVIE,"['history', 'drama']",['US']
...,...,...,...,...,...
2820,ts305329,House Calls with Dr. Phil,SHOW,['reality'],['US']
2821,ts291097,Disrupt & Dismantle with Soledad O'Brien,SHOW,['documentation'],[]
2822,tm1100359,Race Against Time: The CIA and 9/11,MOVIE,['documentation'],[]
2823,tm1097570,Virus Hunting: Cave to COVID,MOVIE,[],[]


### Connect to local database

In [8]:
engine = create_engine('postgresql://postgres:' + password + '@localhost:5432/shows_movies')

In [9]:
inspector = inspect(engine)
inspector.get_table_names()

['credits', 'titles']

In [10]:
# Use pandas to load csv converted DataFrame into database
credits_df.to_sql(name='credits', con=engine, if_exists='append', index=False)
titles_df.to_sql(name='titles', con=engine, if_exists='append', index=False)

In [11]:
# Confirm data has been added by querying the customer_name table
pd.read_sql_query('select * from credits', con=engine).head()

,id,name,role
0,tm19248,Buster Keaton,ACTOR
1,tm19248,Marion Mack,ACTOR
2,tm19248,Glen Cavender,ACTOR
3,tm19248,Jim Farley,ACTOR
4,tm19248,Frederick Vroom,ACTOR


In [12]:
# Confirm data has been added by querying the customer_name table
pd.read_sql_query('select * from titles', con=engine).head()

,id,title,type,genres,production_countries
0,tm19248,The General,MOVIE,"['action', 'drama', 'war', 'western', 'comedy'...",['US']
1,tm83884,His Girl Friday,MOVIE,"['comedy', 'drama', 'romance']",['US']
2,tm19424,Detour,MOVIE,"['thriller', 'drama', 'crime']",['US']
3,tm112005,Marihuana,MOVIE,"['crime', 'drama']",['US']
4,tm22806,Intolerance: Love's Struggle Throughout the Ages,MOVIE,"['history', 'drama']",['US']
